In [111]:
import plotly as py
from plotly import __version__
print(__version__)
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import pandas as pd 
import numpy as np
import plotly.graph_objs as go 
import cufflinks as cf 
import os
import math
#jupyter setup 
init_notebook_mode(connected=True) 
cf.go_offline(connected=True)
cf.set_config_file(theme='white')

%matplotlib inline

3.6.1


In [3]:
file_path = os.path.join('globalpowerplantdatabasev110','global_power_plant_database.csv')
df = pd.read_csv(file_path, sep = ',')
df.head()

,country,country_long,name,gppd_idnr,capacity_mw,latitude,longitude,fuel1,fuel2,fuel3,...,owner,source,url,geolocation_source,year_of_capacity_data,generation_gwh_2013,generation_gwh_2014,generation_gwh_2015,generation_gwh_2016,estimated_generation_gwh
0,AFG,Afghanistan,Kajaki Hydroelectric Power Plant Afghanistan,GEODB0040538,33.00,32.3220,65.1190,Hydro,NaN,NaN,...,NaN,GEODB,http://globalenergyobservatory.org,GEODB,2017.0,NaN,NaN,NaN,NaN,NaN
1,AFG,Afghanistan,Mahipar Hydroelectric Power Plant Afghanistan,GEODB0040541,66.00,34.5560,69.4787,Hydro,NaN,NaN,...,NaN,GEODB,http://globalenergyobservatory.org,GEODB,2017.0,NaN,NaN,NaN,NaN,NaN
2,AFG,Afghanistan,Naghlu Dam Hydroelectric Power Plant Afghanistan,GEODB0040534,100.00,34.6410,69.7170,Hydro,NaN,NaN,...,NaN,GEODB,http://globalenergyobservatory.org,GEODB,2017.0,NaN,NaN,NaN,NaN,NaN
3,AFG,Afghanistan,Nangarhar (Darunta) Hydroelectric Power Plant ...,GEODB0040536,11.55,34.4847,70.3633,Hydro,NaN,NaN,...,NaN,GEODB,http://globalenergyobservatory.org,GEODB,2017.0,NaN,NaN,NaN,NaN,NaN
4,AFG,Afghanistan,Northwest Kabul Power Plant Afghanistan,GEODB0040540,42.00,34.5638,69.1134,Gas,NaN,NaN,...,NaN,GEODB,http://globalenergyobservatory.org,GEODB,2017.0,NaN,NaN,NaN,NaN,NaN


In [6]:
df['year_of_capacity_data'].describe()

count    16145.000000
mean      2016.035367
std          1.258145
min       2000.000000
25%       2016.000000
50%       2016.000000
75%       2016.000000
max       2018.000000
Name: year_of_capacity_data, dtype: float64

In [19]:
 df['fuel1'].iplot(
    kind='hist',
    #bins=30,
    xTitle='major fuel',
    linecolor='black',
    yTitle='count',
    title='Fuel Distribution')

In [26]:
df['capacity_hydro'] = df[df.fuel1 == "Hydro"].capacity_mw
df['capacity_solar'] = df[df.fuel1 == "Solar"].capacity_mw

df[['capacity_hydro', 'capacity_solar']].iplot(
    kind = 'hist',
    linecolor = 'black',
    histnorm = 'percent',
    bins = 30, 
    bargap = .1,
    opacity = 0.8, 
    barmode = 'group',
    xTitle = 'capacity', 
    yTitle = 'percentage',
    title = 'distribution of capacity between two groups')

In [55]:
def binary_fuel(fuel):
    if fuel in ['Gas', 'Oil', 'Coal', 'Petcoke']: 
        return 1
    else: 
        return 0
    
df['envn_unfriend'] = df['fuel1'].apply(binary_fuel)
select = df.loc[:, ['country', 'year_of_capacity_data', 'envn_unfriend', 'capacity_mw']].\
             rename(columns = {'year_of_capacity_data':'year', 'capacity_mw': 'capacity'})

In [82]:
clean2016 = select[select.year==2016.0].groupby(['country', 'envn_unfriend']).capacity.agg(['sum','count'])
clean2016['clean_capacity_pct'] = clean2016.groupby(level=0)['sum'].apply(lambda x: x / float(x.sum()))
clean2016['clean_plant_pct'] = clean2016.groupby(level=0)['count'].apply(lambda x: x / float(x.sum()))
total2016 = select[select.year==2016.0].groupby('country').capacity.agg(['sum','count'])
final = clean2016.loc[:,['clean_capacity_pct', 'clean_plant_pct']].merge(total2016, left_on = 'country', right_on ='country')

In [91]:
final['logsum'] = final['sum'].apply(lambda x:math.log(x))
final = final.reset_index()
final.head()

,country,clean_capacity_pct,clean_plant_pct,sum,count,logsum
0,AUS,1.000000,1.000000,155.000000,2,5.043425
1,CAN,0.814827,0.971014,15779.800000,276,9.666486
2,CAN,0.185173,0.028986,15779.800000,276,9.666486
3,CHL,0.432798,0.612648,18984.771167,253,9.851392
4,CHL,0.567202,0.387352,18984.771167,253,9.851392


In [112]:
final.iplot(y='count', x='logsum', kind='bubble',
                xTitle='total capacity(log)', yTitle='total power plant',
                size='clean_capacity_pct',
                text = ['country','clean_capacity_pct'],
                title='Bubble plot sized by clean power percentage')